In [5]:
torch.cuda.is_available()

True

# Carga del Modelo

In [1]:
HOME_PATH = "/home/223C1851Rodrigo" # path where you cloned musicfm

import os
import sys
import torch

sys.path.append(HOME_PATH)
from musicfm.model.musicfm_25hz import MusicFM25Hz


# dummy audio (30 seconds, 24kHz)
wav = (torch.rand(4, 24000 * 30) - 0.5) * 2

# load MusicFM
musicfm = MusicFM25Hz(
    is_flash=False,
    stat_path=os.path.join(HOME_PATH, "musicfm", "data", "msd_stats.json"),
    model_path=os.path.join(HOME_PATH, "musicfm", "data", "pretrained_msd.pt"),
)

# to GPUs Esto sigue matándo el código
# wav = wav.cuda()
# musicfm = musicfm.cuda()

# get embeddings
musicfm.eval()
emb = musicfm.get_latent(wav, layer_ix=7)
##can expect better performance in downstream tasks by fine-tuning the foundation model. 
##In this scenario, employ musicfm.train() and extract the final embeddings by setting layer_ix=12

/home/223C1851Rodrigo/.local/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/223C1851Rodrigo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Comprobamos que podemos generar los embedings y predicciones de un archivo wav genérico

In [2]:
print( emb.shape)

torch.Size([4, 750, 1024])


In [3]:
print(emb)

tensor([[[ 1.3732,  0.9670, -0.8080,  ...,  0.0794,  0.4502,  0.4360],
         [ 1.1647,  1.0393, -0.2264,  ..., -0.4254, -0.1816, -0.8006],
         [ 1.0308,  1.1129, -0.3464,  ...,  0.6316,  0.3193, -2.6836],
         ...,
         [ 1.2106,  0.6968, -0.1220,  ..., -0.0838, -0.6290, -2.5462],
         [ 1.4991,  0.3841, -0.2471,  ...,  0.0324, -0.0596, -1.4968],
         [ 1.2976,  0.4910, -0.3117,  ..., -0.4541,  0.6745, -0.5085]],

        [[ 1.4553,  1.0022, -0.7665,  ...,  0.4425,  0.2000,  0.3123],
         [ 1.1689,  1.2729, -0.3557,  ...,  0.1232, -0.1057, -0.0860],
         [ 1.3174,  1.1216, -0.1322,  ...,  0.8002,  0.1058, -2.1371],
         ...,
         [ 1.2279,  0.7691,  0.0276,  ...,  0.1213,  0.1203, -3.0150],
         [ 1.6286,  0.5716, -0.2065,  ...,  0.2825,  0.1985, -3.1388],
         [ 1.6812,  0.3665, -0.3592,  ...,  0.1828,  1.1178, -0.1367]],

        [[ 1.3835,  0.9501, -0.8398,  ...,  0.5076,  0.7905,  0.7247],
         [ 1.2815,  1.4665, -0.4137,  ..., -0

In [51]:
print(musicfm.get_predictions(wav))

({'melspec_2048': tensor([[[ -9.5758,  -9.1954,  -1.1780,  ...,  -3.0766,  -1.1205,  -3.4056],
         [ -8.6523,  -9.1994,  -0.5570,  ...,  -4.0734,   0.7417,  -2.8519],
         [ -8.9907,  -8.9420,  -0.3410,  ...,  -4.2216,   0.4235,  -3.1954],
         ...,
         [ -9.1967,  -9.1229,   3.7361,  ...,  -5.1683,  -1.6729,  -4.7571],
         [ -9.4387,  -9.1709,   2.9892,  ...,  -5.3603,  -2.2473,  -5.3881],
         [-10.2053,  -8.8309,  -0.8113,  ...,  -4.6932,  -4.8876,  -5.7795]],

        [[ -9.2164,  -9.1018,  -0.8213,  ...,  -3.1680,  -1.3574,  -3.5422],
         [ -7.7963,  -9.0707,   0.5746,  ...,  -3.9808,   0.3356,  -3.1459],
         [ -8.4450,  -9.3883,   0.9720,  ...,  -4.4189,   0.1756,  -3.2406],
         ...,
         [ -8.2458,  -7.4337,   6.4713,  ...,  -4.3517,  -1.6915,  -3.3164],
         [ -7.7364,  -7.1338,   5.2947,  ...,  -4.6516,  -1.1060,  -3.0578],
         [ -8.1585,  -6.7888,   0.8395,  ...,  -3.8727,  -2.4343,  -2.9385]],

        [[ -9.0656,  -9.49

In [54]:
print(musicfm.get_predictions(wav).shape)

AttributeError: 'tuple' object has no attribute 'shape'

In [33]:
print(wav.shape)

torch.Size([4, 720000])


In [8]:
targets=musicfm.get_targets(wav)
print(targets)

AttributeError: 'MusicFM25Hz' object has no attribute 'quantizer_melspec_2048'

El wav generado automáticamente no tiene targets


In [9]:
print(musicfm.features)

['melspec_2048']


In [ ]:
print(musicfm.modules.features(wav))

In [ ]:
print(musicfm.rearrange(wav))

In [ ]:
print(musicfm.forward(wav))

In [12]:
from torch import nn

# Sequence-level representation
seq_emb = emb.mean(-1) # (batch, time, channel) -> (batch, channel)

# Frame-level representation
"""
	n_frame = desired_temporal_resolution * sequence_length_in_sec
	300 frames = 10Hz * 30s in this example
	As a result, the sequence length becomes from 750 (25Hz * 30s) to 300
"""
n_frame = 300 
token_emb = nn.AdaptiveAvgPool1d(n_frame)(emb) # (batch, time, channel) -> (batch, time', channel)

In [13]:
print(token_emb)

tensor([[[ 0.4122, -0.5057, -0.1025,  ..., -0.2154, -0.3234,  0.4509],
         [ 0.6298,  0.7322,  0.3087,  ..., -0.1382, -0.6659, -0.3734],
         [ 0.5299,  0.1943,  0.3659,  ...,  0.0138, -0.2178, -0.1881],
         ...,
         [ 0.2917,  0.8624, -0.1713,  ..., -0.2558, -0.0072, -0.9173],
         [ 0.2518, -0.2337, -0.2134,  ..., -0.1228, -0.3282, -0.9194],
         [ 0.1638,  0.1603, -0.2724,  ..., -0.1552, -0.9662, -0.5544]],

        [[ 0.3517, -0.7068, -0.2381,  ..., -0.2275, -0.0056,  0.4423],
         [ 0.3921,  0.5885,  0.3864,  ...,  0.0726, -0.2103, -0.7684],
         [ 0.5107,  0.0814,  0.3589,  ...,  0.0689, -0.2204, -0.6979],
         ...,
         [ 0.3807,  0.6826, -0.0566,  ..., -0.1114, -0.0025, -0.5307],
         [ 0.2904,  0.2072, -0.1460,  ..., -0.2535, -0.3225, -1.0727],
         [ 0.3203, -0.1439, -0.1698,  ..., -0.2303, -0.6982, -0.9131]],

        [[ 0.3650, -0.8754, -0.1529,  ..., -0.3190, -0.3571,  0.1070],
         [ 0.4855,  0.7858,  0.3514,  ..., -0

In [14]:
print(len(token_emb[1][1]))

300


In [15]:
print(token_emb[1][1])

tensor([ 3.9206e-01,  5.8855e-01,  3.8636e-01, -1.9816e-01,  2.6726e-01,
        -1.5140e-01, -2.3119e-01, -1.9029e-01, -3.3067e-01,  2.1300e-01,
        -1.7685e-01,  2.8138e-01,  5.0001e-01,  2.8114e-01,  7.4390e-01,
         1.1926e-01,  8.1994e-01,  5.2698e-01,  2.3806e-01,  3.0752e-01,
         8.3861e-01, -2.0649e-01, -2.2014e-01, -1.7771e-01,  5.9291e-01,
         1.6158e-02,  4.5712e-01, -1.1800e-01,  8.0347e-01,  6.6351e-01,
         5.7462e-02, -1.9259e-04,  3.9170e-01,  7.1267e-02,  3.0303e-01,
         1.2654e-01,  5.5030e-01, -1.0599e-01,  3.5775e-02,  4.8415e-01,
         1.4233e-01,  5.0922e-02, -2.2108e-01, -2.0738e-02, -6.4073e-01,
        -6.1304e-01, -4.4317e-01,  1.1809e-01, -5.4358e-01,  9.1369e-02,
         3.7878e-01,  4.9332e-01, -1.3550e-01,  1.9763e-02,  2.7871e-01,
         3.6669e-01, -2.9834e-01, -4.1684e-01,  1.1512e-01,  2.2744e-02,
        -7.9419e-02,  4.4646e-01,  8.4912e-01,  1.6373e-01, -4.5163e-01,
         3.0425e-01, -1.2215e+00, -2.8867e-01,  2.4

In [16]:
print(musicfm.forward)

<bound method MusicFM25Hz.forward of MusicFM25Hz(
  (preprocessor_melspec_2048): MelSTFT(
    (mel_stft): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
    (amplitude_to_db): AmplitudeToDB()
  )
  (quantizer_melspec_2048_0): RandomProjectionQuantizer()
  (conv): Conv2dSubsampling(
    (conv): Sequential(
      (0): Res2dModule(
        (conv1): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv3): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Res2dModule(
        (conv1): Conv2d(512, 512, k

# Intento de cargar un ejemplo

In [66]:
from fma import utils
import numpy
import pandas as pd
import IPython.display as ipd


In [64]:
tracks = utils.load('fma/data/fma_metadata/tracks.csv')
genres = utils.load('fma/data/fma_metadata/genres.csv')
features = utils.load('fma/data/fma_metadata/features.csv')
echonest = utils.load('fma/data/fma_metadata/echonest.csv')

In [67]:
small_tracks = tracks[tracks['set', 'subset'] <= 'small']
print(tracks.shape,small_tracks.shape)

(106574, 52) (8000, 52)


In [4]:
print(small_tracks)

            album                                                              \
         comments        date_created date_released        engineer favorites   
track_id                                                                        
2               0 2008-11-26 01:44:45    2009-01-05             NaN         4   
5               0 2008-11-26 01:44:45    2009-01-05             NaN         4   
10              0 2008-11-26 01:45:08    2008-02-06             NaN         4   
140             1 2008-11-26 01:49:59    2007-05-22             NaN         1   
141             0 2008-11-26 01:49:57    2009-01-16             NaN         1   
...           ...                 ...           ...             ...       ...   
154308          0 2017-03-05 04:57:38    2017-03-05             NaN         0   
154309          0 2017-03-05 04:57:38    2017-03-05             NaN         0   
154413          0 2017-03-07 18:44:11           NaT  Ernie Indradat         0   
154414          0 2017-03-07

In [8]:
small_features=features[tracks['set', 'subset'] <= 'small']
print(small_features.shape)

(8000, 518)


In [59]:
print(small_features.index)

Index([     2,      5,     10,    140,    141,    148,    182,    190,    193,
          194,
       ...
       153956, 154303, 154305, 154306, 154307, 154308, 154309, 154413, 154414,
       155066],
      dtype='int64', name='track_id', length=8000)


In [9]:
##features
print('{1} features for {0} tracks'.format(*features.shape))
columns = ['mfcc', 'chroma_cens', 'tonnetz', 'spectral_contrast']
columns.append(['spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff'])
columns.append(['rmse', 'zcr'])
for column in columns:
    ipd.display(small_features[column].head().style.format('{:.2f}'))

518 features for 106574 tracks


In [71]:
features

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
20           -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   
...                ...       ...       ...       ...       ...       ...   
155316       -0.490129  0.463834  2.321970 -0.084352  1.662914  2.115189   
155317       -0.461559 -0.229601 -0.496632 -0.422033  0.130612 -0.263825   
155318        0.552473 -0.110498 -0.532014  0.263131 -0.224011 -0.530972   
155319       -0.176901  0.187208 -0.050664  0.368843  0.066005 -0.857354   
155320        0.489665  1.862421  0.854461 -0.103666 -0.249835  0.360283   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
20         -0.289294 -0.816410  0.043851 -0.804761  ...  0.095003  0.022492   
...              ...       ...       ...       ...  ...       ...       ...   
155316     -0.237794  5.695442  0.830353  1.951819  ...  0.128410  0.022547   
155317     -0.628103 -0.082687 -0.229483 -0.492753  ...  0.132964  0.023548   
155318      1.713526  1.418444  1.325197  0.120333  ...  0.108324  0.017540   
155319     -0.780860  0.626281 -0.630938 -0.787229  ...  0.088311  0.018328   
155320     -0.366701  0.033578 -0.834606 -1.154845  ...  0.091421  0.020312   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
20          0.021355  16.669037  0.469727  0.047225  0.040039  0.000977   
...              ...        ...       ...       ...       ...       ...   
155316      0.019816   4.448255  0.172852  0.028773  0.028320  0.003906   
155317      0.026527   3.270612  0.196289  0.031116  0.027832  0.002441   
155318      0.020471   2.356727  0.212891  0.038450  0.037109  0.003418   
155319      0.017936   6.188604  0.167480  0.041480  0.038086  0.004883   
155320      0.016794  21.756050  0.845215  0.075141  0.044434  0.004395   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
20          3.189831  0.030993  
...              ...       ...  
155316      0.955388  0.012385  
155317      1.283060  0.019059  
155318      0.828569  0.017904  
155319      1.818740  0.020133  
155320      4.687204  0.137205  

[106574 rows x 518 columns]

In [68]:
tracks

album                                                              \
         comments        date_created date_released engineer favorites     id   
track_id                                                                        
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4      6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2      4   
...           ...                 ...           ...      ...       ...    ...   
155316          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155317          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155318          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155319          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155320          0 2017-03-26 16:22:18    2017-03-26      NaN         1  22906   

                                                                     \
                                                information listens   
track_id                                                              
2                                                   <p></p>    6073   
3                                                   <p></p>    6073   
5                                                   <p></p>    6073   
10                                                      NaN   47632   
20                <p> "spiritual songs" from Nicky Cook</p>    2710   
...                                                     ...     ...   
155316    <p>A live performance at Monty Hall on Feb 17,...    1506   
155317    <p>A live performance at Monty Hall on Feb 17,...    1506   
155318    <p>A live performance at Monty Hall on Feb 17,...    1506   
155319    <p>A live performance at Monty Hall on Feb 17,...    1506   
155320                                                  NaN    7481   

                                                                          ...  \
             producer                                               tags  ...   
track_id                                                                  ...   
2                 NaN                                                 []  ...   
3                 NaN                                                 []  ...   
5                 NaN                                                 []  ...   
10                NaN                                                 []  ...   
20                NaN                                                 []  ...   
...               ...                                                ...  ...   
155316    Monty Hall                                                  []  ...   
155317    Monty Hall                                                  []  ...   
155318    Monty Hall                                                  []  ...   
155319    Monty Hall                                                  []  ...   
155320            NaN  [ballad, epic, rockabilly, curse, hex, hard ro...  ...   

               track                         \
         information interest language_code   
track_id                                      
2                NaN     4656            en   
3                NaN     1470            en   
5                NaN     1933            en   
10               NaN    54881            en   
20               NaN      978            en   
...              ...      ...           ...   
155316           NaN      122           NaN   
155317           NaN      194           NaN   
155318           NaN      214           NaN   
155319           NaN      336           NaN   
155320           NaN      972           NaN   

                                                                              \
        

In [13]:
from pydub import AudioSegment
import torch
import io


src = "fma/data/fma_small/000/000002.mp3"

# Cargar el archivo de audio y exportarlo a WAV 
sound = AudioSegment.from_file(src)  
buffer = io.BytesIO()
wav_1=sound.export(buffer, format="wav")

# Obtener los bytes del buffer
audio_bytes = buffer.getvalue()

# Crear un tensor de PyTorch desde los bytes del audio
audio_tensor = torch.tensor(list(audio_bytes), dtype=torch.float)

print(audio_tensor)

tensor([ 82.,  73.,  70.,  ..., 244., 216., 240.])


In [14]:
print(wav_1)

In [15]:
print(audio_tensor.shape)

torch.Size([5287912])


In [70]:
print(musicfm.get_latent(audio_tensor, layer_ix=7))
##este error loa pañamos luego



RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [128, 22032]

In [39]:
##Probamos ahora convirtiendo el mp3 en un 128-band mel spectrogram:

import librosa
import numpy as np

# Path to your MP3 file
audio_path = "fma/data/fma_small/000/000002.mp3"

# Load the MP3 file
y, sr = librosa.load(audio_path, sr=None) #sr=samplig rate

# Compute the Mel spectrogram
spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)

# Convert to log scale (dB)
spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
print('shape=',spectrogram_db.shape)
print(spectrogram_db)

shape= (128, 2582)
[[-80.       -80.       -56.291954 ... -26.588564 -25.919008 -23.309189]
 [-80.       -74.67524  -58.71791  ... -17.651484 -19.243622 -21.871895]
 [-80.       -62.929466 -53.35289  ... -22.546883 -22.104305 -22.779446]
 ...
 [-80.       -80.       -80.       ... -80.       -80.       -80.      ]
 [-80.       -80.       -80.       ... -80.       -80.       -80.      ]
 [-80.       -80.       -80.       ... -80.       -80.       -80.      ]]


In [19]:
print(wav.type())

torch.FloatTensor


In [22]:
mel_tensor = torch.tensor(spectrogram_db, dtype=torch.float32)


In [23]:
print(mel_tensor)

tensor([[-80.0000, -80.0000, -56.2920,  ..., -26.5886, -25.9190, -23.3092],
        [-80.0000, -74.6752, -58.7179,  ..., -17.6515, -19.2436, -21.8719],
        [-80.0000, -62.9295, -53.3529,  ..., -22.5469, -22.1043, -22.7794],
        ...,
        [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000],
        [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000],
        [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000]])


In [24]:
predicciones=musicfm.get_predictions(mel_tensor)
print(predicciones) 
##no lo ha necesitado poner como un [4,...] dunno if this will be problematic....


({'melspec_2048': tensor([[[ -5.6074,  -6.8333,  -1.1478,  ...,  -4.1277,  -5.5414,  -2.3555],
         [ -6.2324,  -7.6554,  -1.6117,  ...,  -3.3610,  -7.5674,  -1.5994],
         [ -5.4463, -10.9181,  -2.0295,  ...,  -4.4884,  -6.6025,  -1.4875]],

        [[ -5.6398,  -7.3719,  -1.5477,  ...,  -3.4849,  -6.3727,  -2.5927],
         [ -5.4629,  -8.2351,  -1.5753,  ...,  -3.5609,  -7.3190,  -1.7185],
         [ -4.9834, -10.6229,  -2.7347,  ...,  -4.6085,  -6.6319,  -1.3714]],

        [[ -5.4606,  -7.5579,  -1.7619,  ...,  -3.3925,  -6.4056,  -2.7834],
         [ -4.7652,  -8.2257,  -1.3950,  ...,  -3.2495,  -7.2641,  -1.9694],
         [ -4.8360, -10.7265,  -2.3713,  ...,  -4.3620,  -6.5278,  -1.4662]],

        ...,

        [[ -4.3179,  -4.2110,  -3.8921,  ...,  -3.7533,  -3.7986,  -1.6333],
         [ -4.1789,  -5.1182,  -2.0792,  ...,  -2.6079,  -4.5702,  -2.5754],
         [ -7.8525, -13.0927,   0.1940,  ...,  -1.3261,  -5.8678,  -5.7841]],

        [[ -5.0149,  -9.3826,  -7.45

In [31]:
targets_002=musicfm.get_targets(mel_tensor)

EinopsError:  Error while processing rearrange-reduction pattern "b f (t s) -> b t (s f)".
 Input tensor shape: torch.Size([128, 128, 10]). Additional info: {'s': 4}.
 Shape mismatch, can't divide axis of length 10 in chunks of 4

In [32]:
##vamos a pasasrlos a un vector [4.,...]  

##TAMPOCO FUNCIONA

bands_per_group = mel_tensor.size(0) // 4

groups = []

# bandas en grupos de 32 (128/4)
for i in range(4):
    group = mel_tensor[i * bands_per_group:(i + 1) * bands_per_group, :].sum(dim=0)
    groups.append(group)

# agrupar en [4, time_frames]
result_tensor = torch.stack(groups)


print(result_tensor.shape)  

torch.Size([4, 2582])


In [34]:
predicciones_corrected=musicfm.get_predictions(result_tensor)
print(predicciones_corrected) 

({'melspec_2048': tensor([[[-12.0814, -10.7451,  -7.2544,  ...,  -2.1798,  -8.2277,  -4.0904],
         [-12.6770, -11.2610,  -6.8420,  ...,  -2.4335,  -8.1820,  -4.2858],
         [ -7.9508,  -7.9989,  -6.7134,  ...,  -1.4232,  -6.2172,  -3.2715]],

        [[-12.0569, -10.7382,  -7.2613,  ...,  -2.1933,  -8.2331,  -4.0755],
         [-12.6734, -11.2979,  -6.8410,  ...,  -2.4407,  -8.1665,  -4.2991],
         [ -7.9492,  -8.0340,  -6.7000,  ...,  -1.4282,  -6.2302,  -3.3343]],

        [[-12.0405, -10.7764,  -7.1933,  ...,  -2.1917,  -8.2187,  -4.0833],
         [-12.6696, -11.3367,  -6.8581,  ...,  -2.4114,  -8.1791,  -4.2992],
         [ -8.0136,  -8.1155,  -6.7655,  ...,  -1.3698,  -6.3128,  -3.4613]],

        [[-12.0713, -10.7760,  -7.1986,  ...,  -2.1903,  -8.2366,  -4.0900],
         [-12.6889, -11.3004,  -6.8199,  ...,  -2.4207,  -8.1999,  -4.2890],
         [ -8.0431,  -8.0467,  -6.7845,  ...,  -1.3901,  -6.3314,  -3.4394]]],
       grad_fn=<SliceBackward0>)}, (tensor([[[ -6.

In [35]:
targets_corrected=musicfm.get_targets(result_tensor)

EinopsError:  Error while processing rearrange-reduction pattern "b f (t s) -> b t (s f)".
 Input tensor shape: torch.Size([4, 128, 10]). Additional info: {'s': 4}.
 Shape mismatch, can't divide axis of length 10 in chunks of 4

In [37]:
result_tensor.size()


torch.Size([4, 2582])

In [45]:
embedings = musicfm.get_latent(result_tensor, layer_ix=7)
embedings

tensor([[[-1.1773, -0.0797,  0.3099,  ..., -0.3460, -1.4669, -0.0520],
         [-1.1755, -0.0792,  0.2895,  ..., -0.3213, -1.4841, -0.0597],
         [-0.0090, -0.2411,  0.1614,  ..., -0.2633,  0.9471,  0.4811]],

        [[-1.1763, -0.0801,  0.3097,  ..., -0.3455, -1.4664, -0.0521],
         [-1.1781, -0.0795,  0.2895,  ..., -0.3221, -1.4854, -0.0611],
         [-0.0140, -0.2547,  0.1279,  ..., -0.3336,  0.9070,  0.4654]],

        [[-1.1785, -0.0796,  0.3098,  ..., -0.3499, -1.4709, -0.0536],
         [-1.1678, -0.0784,  0.2900,  ..., -0.3221, -1.4839, -0.0574],
         [-0.0417, -0.2657,  0.2344,  ..., -0.3931,  0.8804,  0.4560]],

        [[-1.1836, -0.0803,  0.3120,  ..., -0.3519, -1.4712, -0.0550],
         [-1.1670, -0.0793,  0.2918,  ..., -0.3222, -1.4823, -0.0563],
         [ 0.0047, -0.2446,  0.1716,  ..., -0.3754,  0.8869,  0.4844]]],
       grad_fn=<NativeLayerNormBackward0>)

In [47]:
embedings.size()####porq 3 capas¿?¿?¿?z

torch.Size([4, 3, 1024])

In [62]:
print(small_features[small_features.index==2])

feature    chroma_cens                                                   \
statistics    kurtosis                                                    
number              01        02        03       04        05        06   
track_id                                                                  
2             7.180653  5.230309  0.249321  1.34762  1.482478  0.531371   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   

feature                   zcr                                               \
statistics           kurtosis       max      mean    median  min      skew   
number            06       01        01        01        01   01        

In [ ]:
2	7.180653	5.230309	0.249321	1.347620	1.482478	0.531371	1.481593	2.691455	0.866868	1.341231	...	0.054125	0.012226	0.012111	5.758890	0.459473	0.085629	0.071289	0.000000	2.089872	0.061

In [48]:
print(echonest)

               echonest                                                    \
         audio_features                                                     
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   
...                 ...          ...       ...              ...       ...   
124857         0.007592     0.790364  0.719288         0.853114  0.720715   
124862         0.041498     0.843077  0.536496         0.865151  0.547949   
124863         0.000124     0.609686  0.895136         0.846624  0.632903   

In [17]:
##Otra forma, desde wav tensorflow.io

import tensorflow as tf
import tensorflow_io as tfio

2024-06-30 08:54:12.113265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-30 08:54:13.222560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
src

'fma/data/fma_small/000/000002.mp3'

In [19]:
audio = tfio.audio.AudioIOTensor('fma/data/fma_small/000/000002.mp3')
print(audio)

2024-06-30 08:54:20.158004: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-30 08:54:20.400843: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


<AudioIOTensor: shape=[1321967       2], dtype=<dtype: 'float32'>, rate=44100>


2024-06-30 08:54:20.450596: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-30 08:54:20.450778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-30 08:54:20.453835: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
len(audio)

1321967

In [22]:
audio_slice = audio[100:]


2024-06-30 08:54:53.900608: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at audio_kernels.cc:205 : INVALID_ARGUMENT: read 2643734 from 100 failed: 0
2024-06-30 08:54:53.900644: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: read 2643734 from 100 failed: 0


InvalidArgumentError: {{function_node __wrapped__IO>AudioReadableRead_device_/job:localhost/replica:0/task:0/device:CPU:0}} read 2643734 from 100 failed: 0 [Op:IO>AudioReadableRead]

In [20]:
tensor=audio.to_tensor()

2024-06-30 08:54:32.942527: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at audio_kernels.cc:205 : INVALID_ARGUMENT: read 2643934 from 0 failed: 0
2024-06-30 08:54:32.942562: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: read 2643934 from 0 failed: 0


InvalidArgumentError: {{function_node __wrapped__IO>AudioReadableRead_device_/job:localhost/replica:0/task:0/device:CPU:0}} read 2643934 from 0 failed: 0 [Op:IO>AudioReadableRead] name: 

In [21]:
fade = tfio.audio.fade(audio, fade_in=1000, fade_out=2000, mode="logarithmic")

ValueError: TypeError: Scalar tensor has no `len()`
Traceback (most recent call last):

  File "/home/223C1851Rodrigo/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py", line 349, in __len__
    raise TypeError("Scalar tensor has no `len()`")

TypeError: Scalar tensor has no `len()`



In [10]:
spectrogram = tfio.audio.spectrogram(audio, nfft=512, window=512, stride=256)


ValueError: TypeError: Scalar tensor has no `len()`
Traceback (most recent call last):

  File "/home/223C1851Rodrigo/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py", line 349, in __len__
    raise TypeError("Scalar tensor has no `len()`")

TypeError: Scalar tensor has no `len()`



In [ ]:
mel_spectrogram = tfio.audio.melscale(spectrogram, rate=16000, mels=128, fmin=0, fmax=8000)